In [2]:
import pandas as pd
import sys
sys.path.append("../")
import util.path as paths
import numpy as np

DATAROOT = paths.BAPPS_dataset_path

In [3]:
stlpipsAlex = pd.read_csv("transferableAdv_benchmark/stlpipsAlex_transferableAdv.csv")
ssim = pd.read_csv("transferableAdv_benchmark/ssim_transferableAdv.csv")
l2 = pd.read_csv("transferableAdv_benchmark/l2_transferableAdv.csv")

metrics = [l2, ssim, stlpipsAlex]
metricNames = ["l2", "ssim", "stlpipsAlex"]

In [4]:
h = []
for m in metrics[0].iterrows():
    distortion, id_ = m[1]['sample'].split("_")
    h.append(np.load(DATAROOT + 'val/' + distortion + '/judge/' + id_ + '.npy')[0])

In [8]:
# with open('metricFinal_tmp.txt','w') as f:
for i, metric in enumerate(metrics):
    rank = (metric["d0"]>metric["d1"]).apply(lambda x: int(x))
    metric["rank"] = rank
    metric["h"] = h
    
    toPrint = metricNames[i]+" &"
    
    toPrint += str((metric["rank"]==metric["h"]).sum())+"/"+str(np.round(100.*(metric["rank"]==metric["h"]).sum()/len(metric)).astype(int))+"\% &"

    metric = metric[metric["rank"]==metric["h"]]

    pgd = []
    for row in metric.iterrows():
        if row[1]["rank"]:
            s_other = row[1]["d0"]
        else:
            s_other = row[1]["d1"]
        if row[1]["dPGD10"]>s_other:
            pgd.append(1)
        else:
            pgd.append(0)

    toPrint += str(np.sum(pgd))+"/"+str(np.round(100*np.sum(pgd)/len(pgd)).astype(int))+"\% &"
    
    pgd = []
    for row in metric.iterrows():
        if row[1]["rank"]:
            s_other = row[1]["d0"]
        else:
            s_other = row[1]["d1"]
        if row[1]["dPGD20"]>s_other:
            pgd.append(1)
        else:
            pgd.append(0)

    toPrint += str(np.sum(pgd))+"/"+str(np.round(100*np.sum(pgd)/len(pgd)).astype(int))+"\% &"

    stadv = []
    for row in metric.iterrows():
        if row[1]["rank"]:
            s_other = row[1]["d0"]
        else:
            s_other = row[1]["d1"]
        if row[1]["dstAdv"]>s_other:
            stadv.append(1)
        else:
            stadv.append(0)
    
    toPrint += str(np.sum(stadv))+"/"+str(np.round(100*np.sum(stadv)/len(stadv)).astype(int))+"\% &"

#         ks = [5,10,15,20,25,30]
    # ks = [5,10,15,20]
    ks = [10,20]
    for k in ks:
        stadvpgd = []
        for row in metric.iterrows():
            if row[1]["rank"]:
                s_other = row[1]["d0"]
            else:
                s_other = row[1]["d1"]
            if row[1]["dstAdvPGD"+str(k)]>s_other:
                stadvpgd.append(1)
            else:
                stadvpgd.append(0)
        if k!=20:
            toPrint += str(np.sum(stadvpgd))+"/"+str(np.round(100*np.sum(stadvpgd)/len(stadvpgd)).astype(int))+"\% &"
        else:
            toPrint += str(np.sum(stadvpgd))+"/"+str(np.round(100*np.sum(stadvpgd)/len(stadvpgd)).astype(int))+"\%"
    toPrint += " \\\ \n"

    # f.write(toPrint)
    print(toPrint)

l2 &2099/77\% &101/5\% &174/8\% &77/4\% &189/9\% &257/12\% \\ 

ssim &2093/77\% &237/11\% &442/21\% &78/4\% &339/16\% &540/26\% \\ 

stlpipsAlex &2470/91\% &104/4\% &198/8\% &96/4\% &205/8\% &310/13\% \\ 

